<a href="https://colab.research.google.com/github/AvantiShri/colab_notebooks/blob/master/AdaptDeepSEAvariableInputLength.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget http://mitra.stanford.edu/kundaje/avanti/deepsea_beluga/deepseabeluga_keras.h5 -O deepseabeluga_keras.h5

--2019-10-08 18:05:03--  http://mitra.stanford.edu/kundaje/avanti/deepsea_beluga/deepseabeluga_keras.h5
Resolving mitra.stanford.edu (mitra.stanford.edu)... 171.67.96.243
Connecting to mitra.stanford.edu (mitra.stanford.edu)|171.67.96.243|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 598195356 (570M)
Saving to: ‘deepseabeluga_keras.h5’

deepseabeluga_keras 100%[===================>] 570.48M  14.1MB/s    in 31s     

2019-10-08 18:05:34 (18.6 MB/s) - ‘deepseabeluga_keras.h5’ saved [598195356/598195356]



In [1]:
import keras
from keras.models import load_model

model = load_model("deepseabeluga_keras.h5")
model.summary()

Using TensorFlow backend.



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 1993, 320)         10560     
_________________________________________________________________
activation_1 (Activation)    (None, 1993, 320)         0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1986, 320)         819520    
_________________________________________________________________
activation_2 (Activation)    (None, 1986, 320)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1986, 320)         0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 496, 320)          0         
___

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:310: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [0]:
from keras.models import Sequential
from keras.layers.core import Dense,Flatten
from keras.layers.convolutional import Conv1D

new_model = Sequential()
#Different first layer with input_shape=(400,4) instead of (2000,4)
new_model.add(Conv1D(kernel_size=8, filters=320, input_shape=(400,4)))
new_model.layers[0].set_weights(model.layers[0].get_weights())
#We copy over remaining layers up until the Permute layer, which we are
# going to get rid of the need for (I had the Permute layer to cope
# with the fact that the original pytorch model was in a 'channels first'
# format, but Keras is 'channels last', and the weights of the pytorch Dense
# layer were such that it was expecting the output of the conv layer to
# be in a 'channels first' format - however, we will deal with this by
# just rearranging the weights of the Dense layer, since we have to modify
# those weights anyway).
for layer in model.layers[1:16]:
  layer_config = layer.get_config()
  del layer_config['name'] #let the name be autogenerated to avoid collisions
  new_layer = type(layer)(**layer_config)
  if (len(layer.get_weights()) > 0):
      new_layer.build(layer.input_shape)
  new_layer.set_weights(layer.get_weights())
  new_model.add(new_layer)
new_model.add(Flatten())
new_model.add(Dense(2003,name="altereddense"))
for layer in model.layers[20:]:
  layer_config = layer.get_config()
  del layer_config['name'] #let the name be autogenerated to avoid collisions
  new_layer = type(layer)(**layer_config)
  if (len(layer.get_weights()) > 0):
      new_layer.build(layer.input_shape)
  new_layer.set_weights(layer.get_weights())
  new_model.add(new_layer)

first_dense_layer_weights, first_dense_layer_biases = model.layers[19].get_weights()
#These next lines alter the weights of the dense layer so that the whole model
# will work on an input of length 400bp
#While porting the weights over, we also get rid of the need for the Permute
# layer by applying the 'transpose' on the weights. The truncation of 50:56 is
# what gets the model to work on an input of length 400bp
new_first_dense_layer_weights = first_dense_layer_weights.reshape((640,106,2003))[:,50:56,:].transpose((1,0,2)).reshape((3840,2003))
new_model.layers[17].set_weights([new_first_dense_layer_weights, first_dense_layer_biases])

In [0]:
import numpy as np
random_inputs = np.random.RandomState(1).rand(20,2000,4)

In [0]:
from keras.models import Model
from keras.layers.core import Permute, Flatten

#the output of new_model should be the same as if we took the original model,
# but artificially set the activations on the flanks of the last conv layer
# to zero for neurons with a receptive field that extended outside the central
# 400bp. We test that here.
orig_model_last_conv_out_model = Model(inputs=model.input,
                                 outputs=model.layers[16].output)
last_bit_of_model = Sequential()
last_bit_of_model.add(Permute(dims=(2,1), input_shape=(106,640)))
for layer in model.layers[18:]:
  last_bit_of_model.add(layer)

orig_model_last_conv_out = orig_model_last_conv_out_model.predict(random_inputs)
orig_model_last_conv_out[:,0:50,:] = 0.0
orig_model_last_conv_out[:,56:,:] = 0.0
zerodflanks_orig_model_out = last_bit_of_model.predict(orig_model_last_conv_out)

In [0]:
new_preds = new_model.predict(random_inputs[:,800:1200,:])

In [6]:
np.max(np.abs(zerodflanks_orig_model_out-new_preds))

1.1920929e-07

In [7]:
new_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 393, 320)          10560     
_________________________________________________________________
activation_1 (Activation)    (None, 393, 320)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 386, 320)          819520    
_________________________________________________________________
activation_2 (Activation)    (None, 386, 320)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 386, 320)          0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 96, 320)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 89, 480)          